# Libraries

In [ ]:
#!pip install pymongo
#!pip install yfinance
import pymongo 
from pymongo import MongoClient
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import yfinance as yf
# Ignore all warnings
warnings.filterwarnings("ignore")
# Ignore specific warnings (e.g., DeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# MongoDB

In [ ]:
# Connect to the MongoDB client locally
client = MongoClient()

# Access the database and the collection
db = client["Dstoolsproject"]  
collection = db["Saudistocktdawul"]  

In [ ]:
# Converting to a dataframe
# Fetch data from the collection
data = list(collection.find())  # find all documents and Converts the cursor to a list

# Convert to DataFrame 
df = pd.DataFrame(data)

# Sorting gathering the records of same company then the next company so on , and each company sorting the date in asc order 
df=df.sort_values(by=['symbol', 'date'], ascending=[True, True])
df = df.reset_index(drop=True)

# Display the data
df.head()

In [ ]:
# Drop the column 'id'
df = df.drop(columns=['_id'])    # Not important for our data
# Edit sector name
df.rename(columns={'sectoer': 'sector'}, inplace=True)

In [ ]:
# Display initial info about the dataset
print("Initial dataset information:")   # There is nulls in open , high , low , no of trades
print(df.info())
print("\n\nSummary statistics:")        # we got from this that we have zeros in close and the last three col 
print(df.describe())                    # The data starts from 31-12-2001 to 23-4-2020                                       

### Get numbers of companies and sectors

In [ ]:
# Unique companies
print(df[['symbol', 'trading_name ', 'sector']].drop_duplicates())    # 200 company
   
# Unique sectors
df[['sector']].drop_duplicates()          # 11 sector

# Cleaning and Preprocessing

## Dulpication

In [ ]:
# number of duplicate rows in the dataset
print("Number of duplications: ",df.duplicated().sum())   # 240 duplication

# Drop duplicate rows
df = df.drop_duplicates()

# Reset the index for cleaner indexing after removing duplicates
df = df.reset_index(drop=True)

# Make sure that that each company is recorded only once per day even if not same records
print("Companies has multiple records in same day: ",df.duplicated(subset=['symbol', 'date']).sum(),"\n")   

# Print the DataFrame to verify duplicates are removed
print(df.info())  # The data becomes 600571 instead if 600811

## Missing values and Zeros

In [ ]:
# We know that we have nulls in open , high , low and no of trades

# We observed that there is companies has a large amount of data by zeros and nulls, we will remove these companies

# Get the unique symbols (companies) from the 'symbol' column
unique_symbols = df['symbol'].unique()

# Iterate through each unique symbol
for symbol in unique_symbols:
    # Filter the DataFrame for each symbol
    symbol_data = df[df['symbol'] == symbol]
    
    # Get the trading name
    trading_name = symbol_data['trading_name '].iloc[0] 
    
    # Count the number of rows
    row_count = symbol_data.shape[0]
    
    # Count the number of null values across all columns for this symbol
    null_count = symbol_data.isnull().sum().sum()
    
    # List of specific columns to check for zeros
    columns_to_check = ['open', 'high', 'low', 'close', 'volume_traded ', 'value_traded', 'no_trades ']  

    # Filter rows in symbol_data where any of the specified columns have a zero
    rows_with_zeros = symbol_data[symbol_data[columns_to_check].eq(0).any(axis=1)]
    
    # Print the companies has more than 500 missing values
    if(null_count>500):
        print(f"Symbol: {symbol} - Trading name: {trading_name} -Rows: {row_count}, Nulls: {null_count}")
                                    # 13 company has more than 500 missing values
                                    # 1120 and 1150 missing all trading values , 7040 from 5/2011 to 3/2012 zeros and nulls

    # Check if rows_with_zeros exceed 500
    if rows_with_zeros.shape[0] > 19:
        # Print the result for the company
        print(f"Symbol: {symbol} - Trading name: {trading_name} - Rows with Zeros: {rows_with_zeros.shape[0]}")            

In [ ]:
# Removing the companies that has nulls > 500 or zeros > 19    ==>      (21 company)
# List of symbols to remove
symbols_to_remove = [1120, 1150, 1330, 2100 ,2110, 2170 , 3008, 4012, 4051, 4061, 4070, 4130, 4160, 4191, 6012, 7020, 7040, 7201, 8110, 8150, 8270]

# Filter out rows with these symbols
df = df[~df['symbol'].isin(symbols_to_remove)]
df = df.reset_index(drop=True)


print(df.info(),"\n")   # Now we have nulls in open , high , low only


# First fill na and zeros in open column by the close of previous day 
# The data already the date is arranged but avoid to fill open day in company by close of another company
# fill the first day in company if has null or zero by close - change

for i in range(1, len(df)):
    if df.loc[i, 'symbol'] == df.loc[i - 1, 'symbol']:  # Ensure it's the same symbol
         if pd.isna(df.loc[i, 'open']) or df.loc[i, 'open'] == 0:  # Check for NaN or zero
            df.loc[i, 'open'] = df.loc[i - 1, 'close']  # Fill with previous close
    else:         
         if pd.isna(df.loc[i, 'open']) or df.loc[i, 'open'] == 0:  # Check for NaN or zero
            if df.loc[i,'close']!=0 :    # Ensure that the close not = 0
                df.loc[i, 'open'] = df.loc[i,'close'] - df.loc[i,'change']  # Fill with close - change

            
print(df.info())   # Now we have nulls in high , low only

# Drop all rows where 'open' is NaN
df = df[df['open'].notna()]   # 3 rows
df = df.reset_index(drop=True)

### Get numbers of companies and sectors after drop some companies

In [ ]:
# Unique companies
print(df[['symbol', 'trading_name ', 'sector']].drop_duplicates())    # 179 company 
   
# Unique sectors
df[['sector']].drop_duplicates()                    # Still 11 sector

In [ ]:
# Count rows where 'close' column equals 0
zero_close_count = (df['close'] == 0).sum()
print(f"Number of rows with 'close' == 0: {zero_close_count}")   # 13 rows

# Remove zeros and fill it by open of next day
for i in range(len(df) - 1):  # Skip the last row, as there's no next row for it
    if df.loc[i, 'close'] == 0 and df.loc[i, 'symbol'] == df.loc[i + 1, 'symbol'] :
        # Replace the 'close' with the 'open' of the next row (same symbol group)
        df.loc[i, 'close'] = df.loc[i + 1, 'open']
        
# Count rows where 'close' column equals 0
zero_close_count = (df['close'] == 0).sum()
print(f"Number of rows with 'close' == 0: {zero_close_count}\n") 

# Drop rows where 'close' equals 0
df = df[df['close'] != 0]  # 5 rows
df = df.reset_index(drop=True)

print(df.info())

In [ ]:
# Check for zeros in the specified columns
columns_to_check = ['volume_traded ', 'value_traded', 'no_trades ']

# Iterate over the columns and count zeros
for col in columns_to_check:
    zero_count = (df[col] == 0).sum()  # Count rows where the column is 0
    print(f"Column '{col}' has {zero_count} zeros.")


# List of columns to check and replace zeros
columns_to_check = ['volume_traded ', 'value_traded', 'no_trades ']

# Iterate over the columns
for col in columns_to_check:
    # Replace zeros with the mean of the column within the same symbol group
    df[col] = df.groupby('symbol')[col].transform(lambda x: x.replace(0, x.mean()) if not x.empty else x)

# Replace NaN or zero values in 'high' with the max of 'open' and 'close'
df['high'] = df.apply(lambda row: max(row['open'], row['close']) if pd.isna(row['high']) or row['high'] == 0 else row['high'], axis=1)

# Replace NaN or zero values in 'low' with the min of 'open' and 'close'
df['low'] = df.apply(lambda row: min(row['open'], row['close']) if pd.isna(row['low']) or row['low'] == 0 else row['low'], axis=1)

# Print the updated DataFrame
print(df.info())  # No nulls no zeros

## Outliers

### Boxplots before removing outliers

In [ ]:
# List of numeric columns for which we want to create box plots
numeric_cols = ['open', 'high', 'low', 'close', 'volume_traded ', 'value_traded', 'no_trades ']

# Group the data by 'symbol'
grouped = df.groupby('symbol')

# Create a box plot for each company for each column
for symbol, group in grouped:
    print(f"Box plots for Company Symbol: {symbol}")
    
    # Create a figure for the company
    plt.figure(figsize=(12, 6))
    
    # Iterate over numeric columns and create subplots
    for i, col in enumerate(numeric_cols, 1):
        plt.subplot(2, 4, i)  # Adjust subplot layout (2 rows, 4 columns)
        sns.boxplot(data=group, y=col, palette="Set3")
        plt.title(f"{col} (Symbol: {symbol})")
        plt.ylabel(col)
        plt.xticks([])  # Remove x-axis ticks
    
    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()


### Removing outliers and save in new df

In [ ]:
# Function to remove outliers based on IQR
def remove_outliers(group, cols):
    for col in cols:
        Q1 = group[col].quantile(0.25)  # 25th percentile
        Q3 = group[col].quantile(0.75)  # 75th percentile
        IQR = Q3 - Q1                   # Interquartile range
        lower_bound = Q1 - 1.0 * IQR    # Lower bound
        upper_bound = Q3 + 1.0 * IQR    # Upper bound
        
        # Remove rows where the column value is outside the bounds
        group = group[(group[col] >= lower_bound) & (group[col] <= upper_bound)]
    return group


numeric_cols = ['open', 'high', 'low', 'close', 'volume_traded ', 'value_traded', 'no_trades ']
# Apply the function to remove outliers for each company
df_cleaned = df.groupby('symbol').apply(lambda g: remove_outliers(g, numeric_cols))  # New df => df_cleaned

# Reset the index after removing outliers
df_cleaned.reset_index(drop=True, inplace=True)



# Print information about the cleaned dataset
print(f"Original DataFrame rows: {df.shape[0]}")
print(f"Cleaned DataFrame rows: {df_cleaned.shape[0]}")


### Boxplots after removing outliers

In [ ]:
# Group the DataFrame by 'symbol' (company)
grouped = df_cleaned.groupby('symbol')

# Iterate over each group (company)
for symbol, group in grouped:
    print(f"Box plots for Company Symbol: {symbol}")
    
    # Create a figure for the company
    plt.figure(figsize=(12, 6))  # Adjust the size of the figure
    
    # Iterate over numeric columns and create subplots
    for i, col in enumerate(numeric_cols, 1):
        plt.subplot(2, 4, i)  # Adjust subplot layout (2 rows, 4 columns)
        sns.boxplot(data=group, y=col, palette="Set3")
        plt.title(f"{col} (Symbol: {symbol})")
        plt.ylabel(col)
        plt.xticks([])  # Remove x-axis ticks for cleaner appearance
    
    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()                             # still there is outliers but less than before 


### Adding New columns to df

In [ ]:
# Add a column 'avg_price' to the dataframe
df_cleaned['avg_price'] = (df_cleaned['open'] + df_cleaned['high'] + df_cleaned['low'] + df_cleaned['close']) / 4


# Add a column 'year' to the dataframe
df_cleaned['year'] = df_cleaned['date'].dt.year

# Visualizations and Insights

### Correlation Heatmap

In [ ]:
# Select only numeric columns from the DataFrame
numeric_df = df_cleaned.select_dtypes(include=['number'])
numeric_df = numeric_df.drop(columns=['symbol'])

# Compute the correlation matrix
correlation_matrix = numeric_df.corr()
    
# Create the mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    
# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(13, 10))
    
# Create the heatmap
sns.heatmap(
    correlation_matrix,
    mask=mask,
    cmap=sns.diverging_palette(180, 10, as_cmap=True),
    annot=True,
    fmt='.2f',
    linewidths=0.5,
    ax=ax
)
plt.title('Correlation Heatmap', fontsize=16)
    
# Fix for matplotlib bug that cuts off the top/bottom
b, t = plt.ylim()  # Discover the values for bottom and top
b += 0.5  # Add 0.5 to the bottom
t -= 0.5  # Subtract 0.5 from the top
plt.ylim(b, t)  # Update the ylim(bottom, top) values
    
# Show the plot
plt.show()

### Distribution of sectors in data

In [ ]:
# Group the data by 'sector' and count the occurrences
sector_counts = df_cleaned['sector'].value_counts()

# Use a more vibrant color palette (e.g., "muted" or "deep")
colors = sns.color_palette("deep", len(sector_counts))

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(sector_counts, labels=sector_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)

# external legend due to the labels are too close
plt.legend(sector_counts.index, title="Sector", loc="upper left", bbox_to_anchor=(1, 1))

# Set title and display the plot
plt.title('Distribution of Sector')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

### Number of companies in each sector

In [ ]:
sector_company_counts = df_cleaned.groupby('sector')['name'].nunique().reset_index()
sector_company_counts_sorted = sector_company_counts.sort_values(by='name', ascending=False)
plt.figure(figsize=(14, 8))
sns.barplot(x='name', y='sector', data=sector_company_counts_sorted, palette='viridis')
plt.title('Number of Companies per Sector', fontsize=16)
plt.xlabel('Number of Companies', fontsize=12)
plt.ylabel('Sector', fontsize=12)

plt.tight_layout()
plt.show()

### Mean of numerical data from Saudi Tadawul stock market over the years.

In [ ]:
# Group the data by year and calculate the mean for the relevant columns
yearly_data = df_cleaned.groupby('year')[['open', 'high', 'low', 'close', 'volume_traded ', 'no_trades ', 'value_traded']].mean()

# Plot the changes over the years for each of the relevant columns
plt.figure(figsize=(12, 30))  # Adjusted height to fit all 7 plots

# List of columns to plot
columns_to_plot = ['open', 'high', 'low', 'close', 'volume_traded ', 'no_trades ', 'value_traded']

# Create subplots for each column (arranging them in a column)
for i, column in enumerate(columns_to_plot, 1):
    plt.subplot(7, 1, i)  # 7 rows, 1 column
    plt.plot(yearly_data.index, yearly_data[column], marker='o', linestyle='-', label=column)
    plt.title(f'{column} Over Years')
    plt.xlabel('Year')
    plt.ylabel(f'{column.capitalize()}')
    plt.grid(True)
    plt.legend()

# Adjust layout to avoid overlap
plt.tight_layout()

# Show the plots
plt.show()

### Total trading volume and total trading value for each sector

In [ ]:
# Create the figure and set up the subplots (2 plots side by side)
fig, axes = plt.subplots(1, 2, figsize=(20, 10))  # 1 row, 2 columns with larger size

# First plot: Total Trading Volume Across Sectors
sector_volume_sum = df_cleaned.groupby('sector')['volume_traded '].sum().reset_index()

# Create a bar plot for the summed volume_traded per sector
sns.barplot(x='sector', y='volume_traded ', data=sector_volume_sum, palette="Set2", ax=axes[0])

# Customize the first plot
axes[0].set_title('Total Trading Volume Across Sectors')
axes[0].set_xlabel('Sector')
axes[0].set_ylabel('Total Volume Traded')
axes[0].tick_params(axis='x', rotation=90)  # Rotate sector labels for better visibility
axes[0].grid(True)

# Second plot: Total Trading Value Across Sectors
sector_value_sum = df_cleaned.groupby('sector')['value_traded'].sum().reset_index()

# Create a bar plot for the summed value_traded per sector
sns.barplot(x='sector', y='value_traded', data=sector_value_sum, palette="Set2", ax=axes[1])

# Customize the second plot
axes[1].set_title('Total Trading Value Across Sectors')
axes[1].set_xlabel('Sector')
axes[1].set_ylabel('Total Value Traded')
axes[1].tick_params(axis='x', rotation=90)  # Rotate sector labels for better visibility
axes[1].grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()


### Average price for each sector

In [ ]:
# Calculate average closing price per sector
sector_avg = df_cleaned.groupby('sector')['avg_price'].mean().reset_index()

# Sort by the average close price
sector_avg_sorted = sector_avg.sort_values(by='avg_price', ascending=False)

# Plot a bar chart
plt.figure(figsize=(12, 6))
plt.barh(sector_avg_sorted['sector'], sector_avg_sorted['avg_price'], color='coral')
plt.xlabel('Average Price', fontsize=12)
plt.ylabel('Sector', fontsize=12)
plt.title('Average Price by Sector', fontsize=16)
plt.tight_layout()
plt.show()

### Top 10 companies in Saudi Tadawul stock market

In [ ]:
# Create the figure and set up the subplots (2 plots side by side)
fig, axes = plt.subplots(1, 2, figsize=(20, 15))  # 1 row, 2 columns

# First plot: Top 10 Companies by Trading Volume (volume_traded)
top_companies = df_cleaned.groupby('name')['volume_traded '].sum().sort_values(ascending=False).head(10)

# Get the sectors for the top companies
top_companies_sectors = df_cleaned[df_cleaned['name'].isin(top_companies.index)][['name', 'sector']].drop_duplicates()

# Merge top companies with their sectors
top_companies_with_sector = top_companies.index.to_frame()
top_companies_with_sector['sector'] = top_companies_sectors.set_index('name').loc[top_companies.index, 'sector'].values

# Create labels combining company names and sectors
labels = [f"{name} ({sector})" for name, sector in zip(top_companies_with_sector['name'], top_companies_with_sector['sector'])]

# Plot the first bar chart on the first subplot
top_companies.plot(kind='bar', color='skyblue', ax=axes[0])
axes[0].set_title('Top 10 Companies by Trading Volume')
axes[0].set_xlabel('Company (Sector)')
axes[0].set_ylabel('Total Trading Volume')
axes[0].set_xticks(range(len(top_companies)))
axes[0].set_xticklabels(labels, rotation=90)
axes[0].grid(True)

# Second plot: Top 10 Companies by Value Traded (value_traded)
top_companies = df_cleaned.groupby('name')['value_traded'].sum().sort_values(ascending=False).head(10)

# Get the sectors for the top companies
top_companies_sectors = df_cleaned[df_cleaned['name'].isin(top_companies.index)][['name', 'sector']].drop_duplicates()

# Merge top companies with their sectors
top_companies_with_sector = top_companies.index.to_frame()
top_companies_with_sector['sector'] = top_companies_sectors.set_index('name').loc[top_companies.index, 'sector'].values

# Create labels combining company names and sectors
labels = [f"{name} ({sector})" for name, sector in zip(top_companies_with_sector['name'], top_companies_with_sector['sector'])]

# Plot the second bar chart on the second subplot
top_companies.plot(kind='bar', color='orange', ax=axes[1])
axes[1].set_title('Top 10 Companies by Value Traded')
axes[1].set_xlabel('Company (Sector)')
axes[1].set_ylabel('Total Value Traded')
axes[1].set_xticks(range(len(top_companies)))
axes[1].set_xticklabels(labels, rotation=90)
axes[1].grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()


### Top 10 companies in Materials sector

In [ ]:
# Filter the data for the Materials sector
materials_sector = df_cleaned[df_cleaned['sector'] == 'Materials']

# Group by 'symbol' or 'trading_name', and calculate the sum of 'volume_traded' and 'value_traded'
top_companies = (
    materials_sector.groupby(['symbol', 'name'])[['volume_traded ', 'value_traded']]
    .sum()
    .reset_index()
)

# Sort by 'volume_traded' and get the top 10
top_volume = top_companies.nlargest(10, 'volume_traded ')

# Sort by 'value_traded' and get the top 10
top_value = top_companies.nlargest(10, 'value_traded')

# Plot the bar charts
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Bar plot for top 10 by volume traded
axes[0].bar(top_volume['name'], top_volume['volume_traded '], color='skyblue')
axes[0].set_title('Top 10 Companies by Volume Traded (Materials Sector)')
axes[0].set_ylabel('Volume Traded')
axes[0].tick_params(axis='x', rotation=90, labelsize=10)

# Bar plot for top 10 by value traded
axes[1].bar(top_value['name'], top_value['value_traded'], color='orange')
axes[1].set_title('Top 10 Companies by Value Traded (Materials Sector)')
axes[1].set_ylabel('Value Traded')
axes[1].tick_params(axis='x', rotation=90, labelsize=10)

# Adjust layout
plt.tight_layout()
plt.show()

### Average price of each company in Materials sector

In [ ]:
# Calculate the average price for each company in the Materials sector
avg_price_companies = (
    materials_sector.groupby(['symbol', 'name'])['avg_price']
    .mean()
    .reset_index()
    .sort_values(by='avg_price', ascending=False)
)

# Plot the bar chart
plt.figure(figsize=(14, 10))
plt.bar(avg_price_companies['name'], avg_price_companies['avg_price'], color='lightgreen')
plt.title('Average Price of Companies in Materials Sector', fontsize=16)
plt.xlabel('Company Name', fontsize=12)
plt.ylabel('Average Price', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

# Machine Learning

### Choosing a specific company

In [ ]:
# Get a specific company to perform the model on it
# will choose the company that has high no. of rows

# Get the unique symbols (companies) from the 'symbol' column
unique_symbols = df_cleaned['symbol'].unique()

# Iterate through each unique symbol
for symbol in unique_symbols:
    # Filter the DataFrame for each symbol
    symbol_data = df_cleaned[df_cleaned['symbol'] == symbol]
    
    # Get the trading name
    trading_name = symbol_data['trading_name '].iloc[0] 
    
    # Count the number of rows
    row_count = symbol_data.shape[0]
    
    
    # Print the companies has more than 3400 row
    if(row_count>3400):
        print(f"Symbol: {symbol} - Trading name: {trading_name} -Rows: {row_count}")          

In [ ]:
# Filter the dataframe for company with symbol 3020 (Has the highest amount of data)
company_3020 = df_cleaned[df_cleaned['symbol'] == 3020]
company_3020 = company_3020.reset_index(drop=True)
company_3020

In [ ]:
# first day in data and last day
print("First day: ",company_3020['date'].min())
print("Last day: ",company_3020['date'].max(),'\n')


# Display the updated dataframe with the new column
company_3020

### Correlation Matrix

In [ ]:
# Filter the DataFrame to include only these columns
numeric_cols = ['open', 'high', 'low', 'close', 'volume_traded ', 'value_traded', 'no_trades ','avg_price']
company_3020_numeric = company_3020[numeric_cols]

# Compute the correlation matrix
corr_matrix = company_3020_numeric.corr()

corr_matrix

### Predict close price

In [ ]:
# Independent variables (X) and dependent variable (y)
X = company_3020[['open','volume_traded ']]          # Features 
y = company_3020['close']                            # Target

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the linear regression model
close_model = LinearRegression()
close_model.fit(X_train, y_train)

# Make predictions
y_pred = close_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Coefficients:", close_model.coef_)
print("Model Intercept:", close_model.intercept_)
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)



# Visualize actual vs predicted
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.xlabel("Actual Close Price")
plt.ylabel("Predicted Close Price")
plt.title("Actual vs Predicted Close Price")
plt.show()

### Predict Value of trades

In [ ]:
# Independent variables (X) and dependent variable (y)
X = company_3020[['open','volume_traded ','no_trades ']]    # Features
y = company_3020['value_traded']                            # Target

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the regression model
value_model = LinearRegression()
value_model.fit(X_train, y_train)

# Predictions
y_pred = value_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Coefficients:", value_model.coef_)
print("Model Intercept:", value_model.intercept_)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

                                        # We want to predict value of trades by open price , volume of trades, and
                                        # no. of trades knowing that we dont know the avg price(price is not stable)


# Scraping

In [ ]:
# Define the stock symbol and date range
symbol = "3020.SR"  # stock symbol
start_date = "2020-04-21"
end_date = "2024-12-19"

# Fetch the data
stock_data = yf.download(symbol, start=start_date, end=end_date)

# Convert to DataFrame
realdf = pd.DataFrame(stock_data)

# Display the data
realdf                      # without no of trades and value traded

## Preprocessing and cleaning

In [ ]:
# Remove the 'Ticker' row/column and ensure clean column headers
realdf.columns = realdf.columns.droplevel(0)  # Drop the 'Ticker' 
realdf.reset_index(inplace=True)  # Reset index to make 'Date' a regular column

# Rename the columns to match the desired names
realdf.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

# Ensure 'Date' is treated as a datetime column
realdf['Date'] = pd.to_datetime(realdf['Date'])

# Rearrange columns: swap 'Open' with 'Close' 
realdf = realdf[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

# sort by date
realdf=realdf.sort_values(by=['Date'], ascending=True)

# Display the cleaned DataFrame
realdf                       # 1159 row

In [ ]:
print(realdf.isna().sum(),'\n')   # 0 Nulls

print(realdf.duplicated().sum(),'\n')    # 0 duplications  

print(realdf.info())

# Define a function to remove outliers using the IQR method
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # First quartile (25%)
        Q3 = df[col].quantile(0.75)  # Third quartile (75%)
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Filter out rows with outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Columns to check for outliers
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']

# Remove outliers and save in a new DataFrame
realdf_cleaned = remove_outliers(realdf.copy(), numeric_cols)
realdf_cleaned.reset_index(inplace=True)

# Output the cleaned DataFrame
print("\n\nOutliers removed. Cleaned DataFrame saved in 'realdf_cleaned'.")
# Print information about the cleaned dataset
print(f"Original DataFrame rows: {realdf.shape[0]}")
print(f"Cleaned DataFrame rows: {realdf_cleaned.shape[0]}")

## Adding No_trades and value_traded from excel sheet

In [ ]:
# file path
file_path = r'C:\Users\tsarget\OneDrive\Desktop\Rest of scraping data.xlsx'

# Read the Excel file
complete_realdf = pd.read_excel(file_path)

# Display the updated DataFrame
complete_realdf                     # 215 row

In [ ]:
# Ensure 'Date' is treated as a datetime column
complete_realdf['date'] = pd.to_datetime(complete_realdf['date'])

# sort by date
complete_realdf=complete_realdf.sort_values(by=['date'], ascending=True)

### Merge the two dataframes

In [ ]:
# Set 'Date' as index for both DataFrames
complete_realdf.set_index('date', inplace=True)
realdf_cleaned.set_index('Date', inplace=True)
# Drop the 'Index' column after resetting the index
realdf_cleaned.drop(columns=['index'], inplace=True)

# Add the columns from realdf to complete_realdf
complete_realdf[['open', 'high', 'low', 'close', 'volume']] = realdf_cleaned[['Open', 'High', 'Low', 'Close', 'Volume']]
# Arrange columns
complete_realdf= complete_realdf[['open', 'high', 'low', 'close', 'volume','value_traded','no_trades']] 

# Rename columns in dataframes to be as the main dataframe
complete_realdf.rename(columns={'volume': 'volume_traded ','no_trades': 'no_trades '}, inplace=True)
realdf_cleaned.rename(columns={'Volume': 'volume_traded ', 'Open': 'open'}, inplace=True)

# Reset the index if you need 'Date' back as a column
complete_realdf.reset_index(inplace=True)

complete_realdf

In [ ]:
print(complete_realdf.info())          # There is nulls

# Drop all rows that contain any NaN values
complete_realdf.dropna(inplace=True)
complete_realdf.reset_index(inplace=True)

In [ ]:
# Define a function to remove outliers using the IQR method
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # First quartile (25%)
        Q3 = df[col].quantile(0.75)  # Third quartile (75%)
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Filter out rows with outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Columns to check for outliers
numeric_cols = ['open', 'high', 'low', 'close', 'volume_traded ' ,'value_traded', 'no_trades ']

# Remove outliers and save in a new DataFrame
complete_realdf = remove_outliers(complete_realdf.copy(), numeric_cols)
complete_realdf.reset_index(inplace=True)

# Output the cleaned DataFrame
# Print information about the cleaned dataset
print(f"Cleaned complete DataFrame rows: {complete_realdf.shape[0]}")

# Testing the ML models on the scraping data

In [ ]:
# Extract features from realdf
X_new = realdf_cleaned[['open', 'volume_traded ']]  
# Predict using the trained model
y_new_pred = close_model.predict(X_new)

# Add the predictions to the DataFrame for comparison or further analysis
realdf_cleaned['predicted_close'] = y_new_pred

# Evaluate the model
mse = mean_squared_error(realdf_cleaned['Close'], y_new_pred)
r2 = r2_score(realdf_cleaned['Close'], y_new_pred)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

# Optional: Visualize the predicted vs. actual close prices (if actual close prices are available in realdf)
if 'Close' in realdf.columns:
    plt.figure(figsize=(8, 6))
    plt.scatter(realdf_cleaned['Close'], realdf_cleaned['predicted_close'], alpha=0.6)
    plt.xlabel("Actual Close Price")
    plt.ylabel("Predicted Close Price")
    plt.title("Actual vs Predicted Close Price")
    plt.show()
    

# Display a sample of the DataFrame with predictions
realdf_cleaned[['Close', 'predicted_close']]

In [ ]:
# Extract features from realdf
X_new = complete_realdf[['open', 'volume_traded ','no_trades ']]  
# Predict using the trained model
y_new_pred = value_model.predict(X_new)

# Add the predictions to the DataFrame for comparison or further analysis
complete_realdf['predicted_value'] = y_new_pred

# Evaluate the model
mse = mean_squared_error(complete_realdf['value_traded'], y_new_pred)
r2 = r2_score(complete_realdf['value_traded'], y_new_pred)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)


# Optional: Visualize the predicted vs. actual close prices (if actual close prices are available in realdf)
if 'value_traded' in complete_realdf.columns:
    plt.figure(figsize=(8, 6))
    plt.scatter(complete_realdf['value_traded'], complete_realdf['predicted_value'], alpha=0.6)
    plt.xlabel("Actual value traded")
    plt.ylabel("Predicted value traded")
    plt.title("Actual vs Predicted value traded")
    plt.show()
    

# Display a sample of the DataFrame with predictions
complete_realdf[['value_traded', 'predicted_value']]